# Writing Fast NumPy Functions with Numba
- Numba is an open source project that creates fast functions for NumPy-like data using CPUs, GPUs, or other hardware.
- Numba significantly speed up numerical computations, especially when working with large arrays

Use: 
- install numba
- decorate your functions
    - `@numba.jit`: allows compatibility with Python objects but might be slower
    - `@numba.njit`: Excludes Python objects, offering faster performance but limited functionality (no Python features like dictionaries or lists)

In [4]:
import numpy as np 

# computes the expression (x - y).mean() using a for loop
def mean_distance(x, y):
    if len(x) != len(y):
        raise ValueError("Arrays x and y must have the same length")
    total_sum = 0 # initialize sum
    
    for i in range(len(x)):
        total_sum += x[i] - y[i] 

    mean_dist = total_sum / len(x) # compute the mean
    return mean_dist

# x = [4, 5, 6]
# y = [1, 2, 3]
# result = mean_distance(x, y)
# print("Mean distance:", result)


# this function is slow:
rng = np.random.default_rng(12345)
x = rng.standard_normal(10_000_000)
y = rng.standard_normal(10_000_000)

%timeit mean_distance(x, y)

5.42 s ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit (x-y).mean()
# numpy version is faster

# turn this fn into a compiled numba fn 

42.9 ms ± 675 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
import numba as nb

numba_mean_distance = nb.jit(mean_distance)
%timeit numba_mean_distance(x,y)

22.5 ms ± 556 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# could have used @nb.jit decorator like 
''' 
@nb.jit
def mean_distance(x, y):
    if len(x) != len(y):
        raise ValueError("Arrays x and y must have the same length")
    total_sum = 0 # initialize sum
    
    for i in range(len(x)):
        total_sum += x[i] - y[i] 

    mean_dist = total_sum / len(x) # compute the mean
    return mean_dist
''' 


- numba can't compile all pure python code, but it supports a significant subset of python that is most useful for writing numerical algorithms
- numba is a deep library, supporting different kinds of hardware, modes of compilation, and user extensions.

# Creating Custom numpy.ufunc Objects with Numba
the `numba.vectorize` fn creates compiled NumPy ufuncs, which behave like built-in ufuncs

In [7]:
from numba import vectorize

@vectorize
def nb_add(x, y):
    return x + y

x = np.arange(10)
nb_add(x, x)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [8]:
nb_add.accumulate(x, 0)

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])